### Import necessary libraries

In [8]:
# import google genai library
from langchain_google_genai import ChatGoogleGenerativeAI

# import pydantic strucutre
from pydantic import BaseModel, Field
from typing import Optional, Any, List

# import prompt template and json output parser
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

from dotenv import load_dotenv
load_dotenv()

True

### Load Model

In [9]:
llm = ChatGoogleGenerativeAI(model="models/gemini-2.0-flash")

In [10]:
llm.invoke("Hi there")

AIMessage(content='Hi! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-89390c86-afb0-4bb0-92a5-17fa7654ea70-0', usage_metadata={'input_tokens': 2, 'output_tokens': 10, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})

### Define Pydantic Structure class

In [12]:
class Results(BaseModel):
    name: str = Field(description="Name of the person")
    email: str = Field(description="Email of the person")
    phone_no: int = Field(description="Phone number of the person")
    feedback: str = Field(description="feedback provided by the person")

#### Define Output Parser and Prompt Template

In [13]:
INITIAL_PROMPT = """
Extract the required info from the text:
---
text:
{text}
---
**Format Instructions:** 
`{format_instructions}`
"""

parser = JsonOutputParser(pydantic_object=Results)

In [14]:
prompt_template = PromptTemplate(template=INITIAL_PROMPT, input_variables=["text"], partial_variables={"format_instructions": parser.get_format_instructions()})

### Define Chain and test it

In [15]:
chain = prompt_template | llm | parser

In [24]:
query = {"text": """
I've been meaning to share my experience with this particular product ever since I received it. To begin with, I wasn't sure about how well it would perform, but the first few days of use changed my perspective. Initially, I thought it might not live up to the description on the website, but it's been surprisingly durable. There was a slight hiccup with delivery—my order showed up a bit later than expected, but the packaging was intact, and everything inside was in perfect condition.

A few days into using the product, I realized how practical it actually is, and it's become a part of my daily routine. However, there's always room for improvement, right? While the design is sleek and modern, I feel like it could use a bit more versatility, perhaps in terms of color options. arbaz123@email.com I did consider reaching out to customer support to inquire about availability, but I didn't have to wait long. 91-786543-7890 I found all the answers I needed in the FAQ section, which was really thorough and helpful. Still, I might explore if they offer a more premium version down the line—just to see if there's a noticeable difference.

On another note, I've had experiences with products from other brands that made me wonder if there was something lacking in the quality of the build. But after comparing, I can say this one stands out in terms of performance, despite a few minor issues that I think are fixable. arbaz_khan_987 After trying it out extensively for a few weeks, I'm starting to understand what sets it apart. For example, the ease of use is something I definitely appreciate.

The one thing I think the company could improve on is their shipping process. It wasn't terrible, but faster delivery options would make a big difference, especially since I ordered it during a time when I had a tight schedule. I remember entering my contact details—like my phone number—just to make sure they could reach me in case of any delays. arbaz.khan.me I didn't receive any direct calls, but I kept an eye on the email for updates.

Now, I'm quite happy with the overall experience. Honestly, I would definitely recommend it to a few friends who are interested. A few tweaks here and there, and I think it could be near perfect. That said, I've been happy enough with the purchase to consider getting another item from the brand. 7890-khan-arbaz There's this one model I've been eyeing that seems to have some great reviews too.

"""}
response = chain.invoke(query)
response

{'name': 'Arbaz Khan',
 'email': 'arbaz123@email.com',
 'phone_no': 7865437890,
 'feedback': "The product is durable and practical, becoming a part of my daily routine. The design is sleek but could use more color options. The FAQ section was thorough and helpful. The company could improve on their shipping process with faster delivery options. Overall, I'm happy with the experience and would recommend it to friends."}

In [22]:
print(type(response))

<class 'dict'>
